<a href="https://colab.research.google.com/github/barbosa-renan/estudos-ia/blob/main/apex_bank_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving dialogflow-key.json to dialogflow-key (1).json


In [2]:
!pip install --quiet google-cloud-dialogflow-cx

import os
from google.cloud import dialogflowcx_v3 as cx
from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import AlreadyExists, PermissionDenied, InvalidArgument

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "dialogflow-key.json"

PROJECT_ID = "apex-bank-472223"
LOCATION   = "us-central1"
AGENT_ID   = "6b37498b-1137-4388-a616-e961c8780864"
LANG_CODE  = "pt-BR"

AGENT_PATH   = f"projects/{PROJECT_ID}/locations/{LOCATION}/agents/{AGENT_ID}"
API_ENDPOINT = f"{LOCATION}-dialogflow.googleapis.com"  # endpoint regional obrigatório

In [9]:


INTENTS = [
    ("saldo.consulta", ["Qual é o meu saldo?", "Quanto tenho na conta?", "Ver saldo disponível"]),
    ("extrato.consulta", ["Mostrar extrato", "Extrato dos últimos 30 dias", "Quero ver minhas transações"]),
    ("fatura.consulta", ["Quando vence minha fatura?", "Valor da fatura do cartão", "Qual minha próxima fatura?"]),
    ("limite.consulta", ["Qual meu limite?", "Quanto de limite eu tenho?", "Limite disponível no cartão"]),
    ("cartao.recomendacao", ["Quero um cartão", "Qual cartão você recomenda?", "Cartão sem anuidade"]),
    ("pix.duvida", ["Como faço um PIX?", "Problema com PIX", "Chave PIX não funciona"]),
    ("ted.duvida", ["TED não caiu", "Quando cai uma TED?", "Problema com transferência TED"]),
    ("boleto.2a_via", ["Segunda via do boleto", "Gerar novo boleto", "Meu boleto venceu"]),
    ("fraude.alerta", ["Sofri fraude", "Transação não reconhecida", "Movimentação suspeita"]),
    ("emprestimo.simulacao", ["Quero simular um empréstimo", "Taxa de empréstimo", "Parcelas do empréstimo"]),
    ("investimento.recomendacao", ["Quero investir", "Sugestão de investimento", "Investimento seguro"]),
    ("seguro.recomendacao", ["Quero contratar seguro", "Seguro viagem", "Seguro para meu cartão"]),
    ("humano.escalacao", ["Quero falar com atendente", "Transferir para humano", "Preciso de ajuda humana"]),
    ("cadastro.atualizacao", ["Quero atualizar meus dados", "Trocar endereço", "Mudar telefone cadastrado"]),
    ("conta.encerramento", ["Quero encerrar minha conta", "Cancelar conta", "Fechar conta bancária"]),
    ("ajuda.geral", ["Ajuda", "O que você faz?", "Quais serviços estão disponíveis?"]),
]

def create_intents_bulk():
    client_opts = ClientOptions(api_endpoint=API_ENDPOINT)
    intents_client = cx.IntentsClient(client_options=client_opts)

    try:
        existing = {i.display_name for i in intents_client.list_intents(parent=AGENT_PATH)}
    except (PermissionDenied, InvalidArgument) as e:
        raise SystemExit(
            "Falha ao listar intents.\n"
            f"- PROJECT_ID: {PROJECT_ID}\n- LOCATION: {LOCATION}\n- AGENT_ID: {AGENT_ID}\n"
            "→ Confirme a região do agente e as permissões da Service Account (Dialogflow API Admin/Client).\n"
            f"Detalhes: {e}"
        )

    created, skipped = [], []

    for display_name, phrases in INTENTS:
        if display_name in existing:
            skipped.append(display_name)
            continue

        training = [
            cx.Intent.TrainingPhrase(
                parts=[cx.Intent.TrainingPhrase.Part(text=p)],
                repeat_count=1
            )
            for p in phrases if p.strip()
        ]

        intent = cx.Intent(
            display_name=display_name,
            training_phrases=training,
            priority=500000
        )

        try:
            resp = intents_client.create_intent(parent=AGENT_PATH, intent=intent)
            created.append(resp.display_name)
        except AlreadyExists:
            skipped.append(display_name)
        except InvalidArgument as e:
            print(f"[ERRO] InvalidArgument ao criar '{display_name}': {e}")

    print("✔ Criadas:", created)
    print("↪ Já existiam:", skipped)

create_intents_bulk()


✔ Criadas: ['saldo.consulta', 'extrato.consulta', 'fatura.consulta', 'limite.consulta', 'cartao.recomendacao', 'pix.duvida', 'ted.duvida', 'boleto.2a_via', 'fraude.alerta', 'emprestimo.simulacao', 'investimento.recomendacao', 'seguro.recomendacao', 'humano.escalacao', 'cadastro.atualizacao', 'conta.encerramento', 'ajuda.geral']
↪ Já existiam: []


In [3]:
from google.cloud import dialogflowcx_v3 as cx
from google.api_core.client_options import ClientOptions

API_ENDPOINT = f"{LOCATION}-dialogflow.googleapis.com"
flows_client = cx.FlowsClient(client_options=ClientOptions(api_endpoint=API_ENDPOINT))

parent = f"projects/{PROJECT_ID}/locations/{LOCATION}/agents/{AGENT_ID}"
flows = list(flows_client.list_flows(parent=parent))

print("Flows do agente:")
for f in flows:
    print(f"-", f.display_name, "→", f.name)

FLOW_NAME = f.name

op = flows_client.train_flow(name=FLOW_NAME)
print("Treinando... aguarde.")
op.result()  # espera até concluir
print("Treinamento concluído!")

op = flows_client.train_flow(name=FLOW_NAME)
print("Treinando... aguarde.")
op.result()  # espera até concluir
print("Treinamento concluído!")



Flows do agente:
- Default Start Flow → projects/apex-bank-472223/locations/us-central1/agents/6b37498b-1137-4388-a616-e961c8780864/flows/00000000-0000-0000-0000-000000000000
Treinando... aguarde.
Treinamento concluído!
Treinando... aguarde.
Treinamento concluído!


In [4]:
from google.cloud import dialogflowcx_v3 as cx
from google.api_core.client_options import ClientOptions
from google.protobuf.field_mask_pb2 import FieldMask

# >>> mantém suas constantes já definidas antes:
# PROJECT_ID, LOCATION, AGENT_ID, AGENT_PATH, etc.
API_ENDPOINT = f"{LOCATION}-dialogflow.googleapis.com"

client_opts   = ClientOptions(api_endpoint=API_ENDPOINT)
flows_client  = cx.FlowsClient(client_options=client_opts)
intents_client= cx.IntentsClient(client_options=client_opts)


In [5]:
# Intents + respostas de exemplo (usando DISPLAY NAME como chave)
ROUTES = {
    "saldo.consulta": "Seu saldo atual é R$ 2.450,00.",
    "extrato.consulta": "Últimos lançamentos: Supermercado R$120, Uber R$35, Restaurante R$75.",
    "fatura.consulta": "Sua próxima fatura vence em 25/09 no valor de R$ 1.200,00.",
    "limite.consulta": "Seu limite total é R$ 5.000,00 e o disponível é R$ 2.300,00.",
    "cartao.recomendacao": "Pelo seu perfil, recomendo o Cartão Premium Internacional (lounge e benefícios de viagem).",
    "cartao.bloqueio": "Ok! Vamos bloquear seu cartão. Para segurança, confirme seus dados no app (demonstração).",
    "cartao.desbloqueio": "Você pode desbloquear o cartão no app: Cartões > Desbloquear (demonstração).",
    "cartao.segunda_via": "Solicitei a segunda via do cartão (simulação). Entrega em até 7 dias úteis.",
    "pix.duvida": "Para PIX use o app: Pagamentos > PIX. Limites e chaves em Configurações do PIX.",
    "ted.duvida": "TED feita até 17h cai no mesmo dia útil. Se não caiu, posso verificar (simulação).",
    "boleto.2a_via": "Gerei a segunda via do boleto (simulação). Veja em Pagamentos > Boletos.",
    "comprovante.enviar": "Enviei o comprovante para o seu e-mail cadastrado (simulação).",
    "fraude.alerta": "Alertei nosso time de segurança (simulação) e recomendo bloqueio preventivo do cartão.",
    "contestar.transacao": "Abertura de contestação registrada (simulação). Retorno em até 10 dias úteis.",
    "senha.redefinir": "Redefina a senha no app: Perfil > Segurança > Redefinir senha.",
    "investimento.recomendacao": "Sugestão: CDB com liquidez diária (baixo risco) para reserva de emergência.",
    "seguro.recomendacao": "Seguro Viagem cobre imprevistos no exterior; posso simular (demonstração).",
    "emprestimo.simulacao": "Posso simular um empréstimo com parcelas flexíveis (demonstração).",
    "humano.escalacao": "Certo! Vou transferir para um atendente humano (simulação).",
    "atendimento.reclamacao": "Registrei sua reclamação (simulação). Vamos acompanhar e retornar.",
    "atendimento.elogio": "Obrigado pelo feedback! Vou repassar ao time. 😊",
    "cadastro.atualizacao": "Você pode atualizar seus dados pelo app: Perfil > Meus dados.",
    "agencia.horario": "Atendimento das 10h às 16h (dias úteis). Confira a agência mais próxima no app.",
    "tarifas.duvida": "A tabela de tarifas está disponível no app: Perfil > Documentos > Tarifas.",
    "viagem.aviso": "Aviso de viagem registrado (simulação). Uso internacional habilitado.",
    "cambio.internacional": "A cotação varia ao longo do dia. Consulte ‘Câmbio’ no app.",
    "lgpd.privacidade": "Tratamos seus dados conforme a LGPD. Você pode solicitar acesso/remoção pelo app.",
    "openfinance.consentimento": "No Open Finance você compartilha dados com consentimento. Posso iniciar (simulação).",
    "conta.encerramento": "Posso iniciar o processo de encerramento (simulação). Confirmar?",
    "ajuda.geral": "Posso ajudar com saldo, extrato, fatura, limites, cartão, PIX/TED e mais."
}

def add_routes():
    # 1) Mapear display_name -> resource name das intents
    intents_map = {i.display_name: i.name for i in intents_client.list_intents(parent=AGENT_PATH)}

    # 2) Buscar o flow atual e saber quais intents já têm route
    flow = flows_client.get_flow(name=FLOW_NAME)
    existing_intent_names = set(r.intent for r in flow.transition_routes if r.intent)

    added = []
    for display_name, resposta in ROUTES.items():
        intent_name = intents_map.get(display_name)
        if not intent_name:
            print(f"[AVISO] Intent não encontrada no agente: {display_name} (pulei)")
            continue
        if intent_name in existing_intent_names:
            # já existe route apontando pra essa intent
            continue

        route = cx.TransitionRoute(
            intent=intent_name,  # <- usar o RESOURCE NAME!
            trigger_fulfillment=cx.Fulfillment(
                messages=[cx.ResponseMessage(text=cx.ResponseMessage.Text(text=[resposta]))]
            )
        )
        flow.transition_routes.append(route)
        added.append(display_name)

    if added:
        flows_client.update_flow(
            flow=flow,
            update_mask=FieldMask(paths=["transition_routes"])
        )
    print("✔ Routes criadas:", added if added else "nenhuma (todas já existiam)")

add_routes()


[AVISO] Intent não encontrada no agente: cartao.bloqueio (pulei)
[AVISO] Intent não encontrada no agente: cartao.desbloqueio (pulei)
[AVISO] Intent não encontrada no agente: cartao.segunda_via (pulei)
[AVISO] Intent não encontrada no agente: comprovante.enviar (pulei)
[AVISO] Intent não encontrada no agente: contestar.transacao (pulei)
[AVISO] Intent não encontrada no agente: senha.redefinir (pulei)
[AVISO] Intent não encontrada no agente: atendimento.reclamacao (pulei)
[AVISO] Intent não encontrada no agente: atendimento.elogio (pulei)
[AVISO] Intent não encontrada no agente: agencia.horario (pulei)
[AVISO] Intent não encontrada no agente: tarifas.duvida (pulei)
[AVISO] Intent não encontrada no agente: viagem.aviso (pulei)
[AVISO] Intent não encontrada no agente: cambio.internacional (pulei)
[AVISO] Intent não encontrada no agente: lgpd.privacidade (pulei)
[AVISO] Intent não encontrada no agente: openfinance.consentimento (pulei)
✔ Routes criadas: ['saldo.consulta', 'extrato.consulta'